Смирнов Николай 

Возьмем данные из файла

In [11]:
import pandas as pd
import re

# Открываем файл SQL и читаем его содержимое
file_path = "outlets.sql"  # Укажите путь к вашему файлу
with open(file_path, "r", encoding="utf-8") as file:
    sql_content = file.read()

# Регулярное выражение для извлечения данных из команды INSERT INTO
insert_pattern = r"INSERT INTO `outlets` \(.*?\) VALUES\s*(.+);"
matches = re.search(insert_pattern, sql_content, re.DOTALL)

if matches:
    data_block = matches.group(1)  # Извлечённый блок данных

    # Преобразуем данные в список кортежей
    rows = re.findall(r"\((.*?)\),?", data_block)
    data = []
    for row in rows:
        # Убираем кавычки и разделяем значения, учитывая запятые внутри кавычек
        values = re.split(r",(?=(?:[^']*'[^']*')*[^']*$)", row)
        values = [value.strip().strip("'") for value in values]
        data.append(values)

    # Проверяем количество колонок в данных
    max_columns = max(len(row) for row in data)
    columns = ["id", "Город дистрибьютора", "Торг_точка_грязная", "Торг_точка_грязная_адрес", "outlet_clean_id"]

    # Добавляем временные заголовки для дополнительных колонок, если их больше
    if max_columns > len(columns):
        columns.extend([f"extra_col_{i}" for i in range(1, max_columns - len(columns) + 1)])

    # Создаём DataFrame
    df = pd.DataFrame(data, columns=columns[:max_columns])

    # Преобразуем значения NULL в None
    df = df.replace("NULL", None)

    #print(df)
else:
    print("Данные не найдены в файле.")

Функция для проверки совпадения 

In [12]:
def is_in(arr1, arr2):
    return str(arr1)[1:-1] in str(arr2)

Функция, чтобы упростить текст

In [13]:
def simple(a):
    if isinstance(a, pd.Series):  # Если передан pandas.Series
        return a.str.replace(r"[\s.,\"'+-]", "", regex=True).str.lower()
    elif isinstance(a, str):  # Если передана строка
        import re
        return re.sub(r"[\s.,\"'+-]", "", a).lower()
    else:
        raise TypeError("Функция simple ожидает pandas.Series или строку.")

In [14]:
data = df

Вынесем записи, где указан, адрес на верх таблицы, чтобы в таблицу без дубликатов попадали именно они

In [15]:
# Сортировка через цикл
valid_rows = []
invalid_rows = []
for _, row in data.iterrows():
    if row["Торг_точка_грязная_адрес"] not in ['он же', '-', None]:
        valid_rows.append(row)
    else:
        invalid_rows.append(row)

# Объединение обратно в итоговый DataFrame
data = pd.DataFrame(valid_rows + invalid_rows).reset_index(drop=True)


Пустая таблица без дубликатов

In [16]:
# Создаем список колонок
columns = [
    "id", 
    "Город дистрибьютора", 
    "Торг_точка_грязная", 
    "Торг_точка_грязная_адрес", 
    "outlet_clean_id",
    "extra_col_1", 
    "extra_col_2", 
    "extra_col_3", 
    "extra_col_4"
]

# Создаем пустую таблицу (DataFrame)
unique_df = pd.DataFrame(columns=columns)

Функция самой процедуры

In [17]:
import pandas as pd

def f(data, unique_df):
    new_rows = []  # Для накопления новых строк
    outlet_clean_id_list = data["outlet_clean_id"].tolist()  # Оптимизируем доступ к данным

    for i, row in enumerate(data.itertuples(index=False)):
        if outlet_clean_id_list[i] is None:
            a = simple(str(row.Торг_точка_грязная))

            new_rows.append(row)  # Добавляем строку в список
            outlet_clean_id_list[i] = row.id  # Обновляем список

            flag = False
            for j in range(i + 1, len(data)):
                if outlet_clean_id_list[j] is None:
                    b = simple(str(data.at[j, "Торг_точка_грязная"]))
                    if is_in(a, b) or is_in(b, a):
                        outlet_clean_id_list[j] = row.id
                        flag = True
            
            if not flag:
                outlet_clean_id_list[i] = None  # По условию задачи

    # Обновляем `data` одним присваиванием
    data["outlet_clean_id"] = outlet_clean_id_list

    # Объединяем `unique_df` одним вызовом `concat`
    if new_rows:
        unique_df = pd.concat([unique_df, pd.DataFrame(new_rows)], ignore_index=True)

    return data, unique_df


In [18]:
a, b = f(data, unique_df)

In [19]:
a[:15]

,id,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,extra_col_1,extra_col_2,extra_col_3,extra_col_4
0,1,Ростов-на-Дону,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",1,None,None,None,None
1,6,Ростов-на-Дону,Алексеева Л П ИП,Буденного 161,6,None,None,None,None
2,8,Ростов-на-Дону,ИП Алиева Т.М.,Ип Алиева Т.М. п.Газетный 39,8,None,None,None,None
3,10,Ростов-на-Дону,Алпатова В В ИП,ул Гвардейская 34-а,10,None,None,None,None
4,17,Ростов-на-Дону,Анохин С М ИП,Ростов на Дону ул Калинина 10-А,17,None,None,None,None
5,22,Ростов-на-Дону,ИП Арустамян,Ростовская обл. г.Морозовск ул. Ворошилова 364/2,22,None,None,None,None
6,28,Ростов-на-Дону,Бага,Ворошиловский 40,28,None,None,None,None
7,31,Ростов-на-Дону,ИП Безбородов,до 12часов,None,None,None,None,None
8,35,Ростов-на-Дону,Беник пок.,Ж,35,None,None,None,None
9,41,Ростов-на-Дону,Боря Дубров,г Азов центр рынок,41,None,None,None,None


In [20]:
b[:15]

,id,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id,extra_col_1,extra_col_2,extra_col_3,extra_col_4,_1
0,1,NaN,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",None,None,None,None,None,Ростов-на-Дону
1,6,NaN,Алексеева Л П ИП,Буденного 161,None,None,None,None,None,Ростов-на-Дону
2,8,NaN,ИП Алиева Т.М.,Ип Алиева Т.М. п.Газетный 39,None,None,None,None,None,Ростов-на-Дону
3,10,NaN,Алпатова В В ИП,ул Гвардейская 34-а,None,None,None,None,None,Ростов-на-Дону
4,17,NaN,Анохин С М ИП,Ростов на Дону ул Калинина 10-А,None,None,None,None,None,Ростов-на-Дону
5,22,NaN,ИП Арустамян,Ростовская обл. г.Морозовск ул. Ворошилова 364/2,None,None,None,None,None,Ростов-на-Дону
6,28,NaN,Бага,Ворошиловский 40,None,None,None,None,None,Ростов-на-Дону
7,31,NaN,ИП Безбородов,до 12часов,None,None,None,None,None,Ростов-на-Дону
8,35,NaN,Беник пок.,Ж,None,None,None,None,None,Ростов-на-Дону
9,41,NaN,Боря Дубров,г Азов центр рынок,None,None,None,None,None,Ростов-на-Дону
